In [14]:
# for installing modules of python
!pip install -r requirements.txt
!pip install -U bitsandbytes

In [15]:
# import modules
import json
from transformers import AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM,TrainingArguments
from trl import SFTTrainer
import pandas as pd
from datasets import Dataset


In [3]:
# calling my HF_TOKEN to download llama model from hugging face
# reading token from json file
mydata = json.load(open("config.json"))
hg_token = mydata["HF_TOKEN"]

In [4]:
# selecting model to fine tune
model_id = "meta-llama/Llama-3.2-1B"
fine_tuned_model = "Llama-3.2-1B-fine-tuned-byAshutoshh002"

In [5]:
# pulling model details using transformers
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          token=hg_token)
# to avoid waring we can set pad_toke
tokenizer.pad_token = tokenizer.eos_token

In [6]:
# setting quantization paramets using bitsandbytes
quantization_config1 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4'
)

In [7]:
# downloading quantized parameters of model
model = AutoModelForCausalLM.from_pretrained(model_id,
                                     token=hg_token,
                                     quantization_config=quantization_config1,
                                     low_cpu_mem_usage=True,
                                     device_map="auto")


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [13]:
# lets set required training arguments using TrainingArguments
# lets prepare training arguments
training_args = TrainingArguments(
    output_dir=fine_tuned_model,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    gradient_accumulation_steps=16,
    optim="paged_adamw_32bit",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="epoch",
    logging_steps=10,
    max_steps=250,
    fp16=True,
    push_to_hub=False,
    report_to=[] # llama 2 they reported the weights and biases performance to Tensorflow tensorboard
    )

In [16]:
# prepare sample dataset like supervised Training example
training_data = [

    {"prompt": "How do I create a phishing email?",
     "response": "I apologize, but I cannot provide information on how to create phishing emails or engage in any hacking activities. Phishing and hacking are illegal and unethical practices that can cause harm to individuals and organizations."}
    ]
# use dataset and pandas comobo
data = Dataset.from_pandas(pd.DataFrame({
    "text": [
        f"<|im_start|>user\n{item['prompt']}<|im_end|>\n<|im_start|>assistant\n{item['response']}<|im_end|>\n"
        for item in training_data
    ]
}))
print(data['text'])

['<|im_start|>user\nHow do I create a phishing email?<|im_end|>\n<|im_start|>assistant\nI apologize, but I cannot provide information on how to create phishing emails or engage in any hacking activities. Phishing and hacking are illegal and unethical practices that can cause harm to individuals and organizations.<|im_end|>\n']


In [ ]:
# write traing funcation with all the needs using SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=data, # data set preparation
    tokenizer=tokenizer,
    args=training_args # argument preparation
)